## 1. 데이터 불러와서 변수설정

In [1]:
import pandas as pd, numpy as np
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

df = pd.read_csv("C:/baf/final_data1102.csv")
df["duration_years"] = df["days_tenure"] / 365.25
df["event"] = df["churn"].astype(int)
df["cust_orig_year"] = pd.to_datetime(df["cust_orig_month"]).dt.year

In [2]:
# 변수설정
selected_vars = [
    "tenure_years", "cpi_at_orig", "gdp_at_orig", "length_of_residence",
    "college_degree", "age_in_years", "has_children", "premium_to_income",
    "sp500_at_orig", "curr_ann_amt", "dgs10_at_orig", "age_at_orig",
    "home_owner", "unrate_at_orig", "good_credit"
]
selected_vars = [c for c in selected_vars if c in df.columns]

## 2. 학습시나리오 설정

In [3]:
scenarios = {
    "3yr": (2014, 2016),
    "5yr": (2012, 2016),
    "7yr": (2010, 2016),
    "9yr": (2008, 2016),
    "11yr": (2006, 2016),
}

results = []

## 3. 학습 루프
##### 루프이므로 나눌 수없어 각 파트별로 주석으로 설명

In [ ]:
for label, (start_yr, end_yr) in scenarios.items():
    print(f"\n {label} 모델 학습 ({start_yr}–{end_yr})")
    
    # 3.1 데이터 분리
    train_df = df[(df["cust_orig_year"] >= start_yr) & (df["cust_orig_year"] <= end_yr)]
    test_df  = df[df["cust_orig_year"].isin([2017, 2018])]
    
    # 3.2 결측치제거 및 표준화
    imp = SimpleImputer(strategy="median")
    scl = StandardScaler()
    
    X_train = pd.DataFrame(imp.fit_transform(train_df[selected_vars]),
                           columns=selected_vars, index=train_df.index)
    X_test  = pd.DataFrame(imp.transform(test_df[selected_vars]),
                           columns=selected_vars, index=test_df.index)
    
    X_train_std = pd.DataFrame(scl.fit_transform(X_train),
                               columns=selected_vars, index=train_df.index)
    X_test_std  = pd.DataFrame(scl.transform(X_test),
                               columns=selected_vars, index=test_df.index)
    
    train_for_cox = pd.concat([train_df[["duration_years","event"]], X_train_std], axis=1)
    test_for_cox  = pd.concat([test_df[["duration_years","event"]], X_test_std], axis=1)
    
    # 3.3 Cox 모델 학습
    cph = CoxPHFitter(penalizer=0.5, l1_ratio=0.0)
    cph.fit(train_for_cox, duration_col="duration_years", event_col="event")
    
    # 3.4 test/ train 에대한 C-index 계산
    train_pred = cph.predict_partial_hazard(train_for_cox)
    test_pred  = cph.predict_partial_hazard(test_for_cox)
    c_train = concordance_index(train_for_cox["duration_years"], -train_pred, train_for_cox["event"])
    c_test  = concordance_index(test_for_cox["duration_years"], -test_pred, test_for_cox["event"])
    
    # 3.5 생존확률 (해지확률) 계산
    time_grid = np.linspace(0, 20, 400)
    S_mat = cph.predict_survival_function(test_for_cox, times=time_grid)
    years = [1,3,5,10]
    churn_means = {}
    for yr in years:
        idx = np.abs(time_grid - yr).argmin()
        churn_means[f"churn_{yr}yr_mean"] = (1 - S_mat.iloc[idx,:]).mean()
    
    results.append({
        "period": label,
        "train_years": f"{start_yr}-{end_yr}",
        "C_train": c_train,
        "C_test": c_test,
        **churn_means
    })

# 3️.5 결과 요약
result_df = pd.DataFrame(results)
print(result_df.round(4))


 3yr 모델 학습 (2014–2016)

 5yr 모델 학습 (2012–2016)

 7yr 모델 학습 (2010–2016)

 9yr 모델 학습 (2008–2016)

 11yr 모델 학습 (2006–2016)
  period train_years  C_train  C_test  churn_1yr_mean  churn_3yr_mean  \
0    3yr   2014-2016   0.9701  0.9336          0.0001          0.0001   
1    5yr   2012-2016   0.9789  0.9467          0.0001          0.0001   
2    7yr   2010-2016   0.9788  0.9205          0.0000          0.0000   
3    9yr   2008-2016   0.9534  0.9107          0.0000          0.0000   
4   11yr   2006-2016   0.9737  0.9068          0.0000          0.0000   
5    3yr   2014-2016   0.9701  0.9336          0.0001          0.0001   
6    5yr   2012-2016   0.9789  0.9467          0.0001          0.0001   
7    7yr   2010-2016   0.9788  0.9205          0.0000          0.0000   
8    9yr   2008-2016   0.9534  0.9107          0.0000          0.0000   
9   11yr   2006-2016   0.9737  0.9068          0.0000          0.0000   

   churn_5yr_mean  churn_10yr_mean  
0          0.0001           0.9870  
1

## 4. 두 모델 검정

#### H0: 두 모델의 예측의 일관성(순위정확도)은 같다

### 4.1 3개년 (2014~ 2016) 모델학습

In [6]:
train_df_3 = df[(df["cust_orig_year"] >= 2014) & (df["cust_orig_year"] <= 2016)]
test_df  = df[df["cust_orig_year"].isin([2017, 2018])]

imp = SimpleImputer(strategy="median")
scl = StandardScaler()

X_train_3 = pd.DataFrame(imp.fit_transform(train_df_3[selected_vars]),
                         columns=selected_vars, index=train_df_3.index)
X_test = pd.DataFrame(imp.transform(test_df[selected_vars]),
                      columns=selected_vars, index=test_df.index)

X_train_std_3 = pd.DataFrame(scl.fit_transform(X_train_3),
                             columns=selected_vars, index=train_df_3.index)
X_test_std = pd.DataFrame(scl.transform(X_test),
                          columns=selected_vars, index=test_df.index)

train_for_cox_3 = pd.concat([train_df_3[["duration_years","event"]], X_train_std_3], axis=1)
test_for_cox = pd.concat([test_df[["duration_years","event"]], X_test_std], axis=1)

### 4.2 5개년 (2012 ~ 2016) 모델 학습

In [7]:
from lifelines import CoxPHFitter

cph_3yr = CoxPHFitter(penalizer=0.5, l1_ratio=0.0)
cph_3yr.fit(train_for_cox_3, duration_col="duration_years", event_col="event")

# 5개년 학습모델
train_df_5 = df[(df["cust_orig_year"] >= 2012) & (df["cust_orig_year"] <= 2016)]

X_train_5 = pd.DataFrame(imp.fit_transform(train_df_5[selected_vars]),
                         columns=selected_vars, index=train_df_5.index)
X_train_std_5 = pd.DataFrame(scl.fit_transform(X_train_5),
                             columns=selected_vars, index=train_df_5.index)
train_for_cox_5 = pd.concat([train_df_5[["duration_years","event"]], X_train_std_5], axis=1)

cph_5yr = CoxPHFitter(penalizer=2.0, l1_ratio=0.3)
cph_5yr.fit(train_for_cox_5, duration_col="duration_years", event_col="event")

<lifelines.CoxPHFitter: fitted with 293275 total observations, 272731 right-censored observations>

### 4.3 test 데이터 셋

In [8]:
test_df  = df[df["cust_orig_year"].isin([2017, 2018])]
X_test  = pd.DataFrame(imp.transform(test_df[selected_vars]), columns=selected_vars, index=test_df.index)
X_test_std = pd.DataFrame(scl.transform(X_test), columns=selected_vars, index=test_df.index)
test_for_cox = pd.concat([test_df[["duration_years","event"]], X_test_std], axis=1)

# 두 모델의 예측 위험도 (partial hazard)
test_pred_3yr = cph_3yr.predict_partial_hazard(test_for_cox).values
test_pred_5yr = cph_5yr.predict_partial_hazard(test_for_cox).values

# 실제 생존정보
durations = test_for_cox["duration_years"].values
events = test_for_cox["event"].values

### 4.4 검정 진행
##### 두 Cox 모델의 Concordance Index 차이(ΔC = C1 - C2)를 Bootstrap으로 검정
##### 귀무가설: C1 = C2 (두 모델의 예측 일관성 동일)
##### 대립가설: C1 ≠ C2 (예측 일관성이 다름)

In [9]:
import numpy as np
from lifelines.utils import concordance_index

def bootstrap_cindex_test(model1_preds, model2_preds, durations, events, n_boot=1000, random_state=42):

    np.random.seed(random_state)
    n = len(durations)
    diffs = []

    for _ in range(n_boot):
        idx = np.random.choice(n, n, replace=True)
        c1 = concordance_index(durations[idx], -model1_preds[idx], events[idx])
        c2 = concordance_index(durations[idx], -model2_preds[idx], events[idx])
        diffs.append(c1 - c2)

    diffs = np.array(diffs)
    mean_diff = np.mean(diffs)
    ci_lower, ci_upper = np.percentile(diffs, [2.5, 97.5])
    p_value = 2 * min(np.mean(diffs > 0), np.mean(diffs < 0))  # 양측검정

    return mean_diff, (ci_lower, ci_upper), p_value, diffs

In [10]:
mean_diff, ci, p, diffs = bootstrap_cindex_test(
    model1_preds=test_pred_3yr,
    model2_preds=test_pred_5yr,
    durations=durations,
    events=events,
    n_boot=500
)

print(f"{mean_diff:.4f}")
print(f"{ci[0]:.4f}, {ci[1]:.4f}")

-0.0141
-0.0155, -0.0128


#### 참고: 그냥 돌리면 검정하는데 매우 오래 걸림 -> 바로위 코드에서 n_boost=200 정도로 낮추면 시간 단축가능하므로, 간단히 확인할때 수정하여 실행